In [ ]:
import matplotlib.pyplot as plt
import os
import random
import sys
import json
import datetime
# import numpy as np
import skimage.io
from imgaug import augmenters as iaa
import glob, pylab, pandas as pd
import pydicom, numpy as np
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
from sklearn.model_selection import KFold

In [ ]:
DATA_DIR = '/kaggle/input/rsna-pneumonia-detection-challenge'
ROOT_DIR = '/kaggle/working'
train_dir=os.path.join(DATA_DIR, 'stage_2_train_images')
test_dir=os.path.join(DATA_DIR, 'stage_2_test_images')

In [ ]:
!git clone https://www.github.com/matterport/Mask_RCNN.git


In [ ]:
# Import Mask RCNN
os.chdir('Mask_RCNN')
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
COCO_WEIGHTS_PATH = "mask_rcnn_coco.h5"

In [ ]:
def parse_data(df):
    """
    Method to read a CSV file (Pandas dataframe) and parse the 
    data into the following nested dictionary:

      parsed = {
        
        'patientId-00': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        },
        'patientId-01': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        }, ...

      }

    """
    # --- Define lambda to extract coords in list [y, x, height, width]
    extract_box = lambda row: [row['y'], row['x'], row['height'], row['width']]

    parsed = {}
    for n, row in df.iterrows():
        # --- Initialize patient entry into parsed 
        pid = row['patientId']
        if pid not in parsed:
            parsed[pid] = {
                'dicom': train_dir+ '/%s.dcm' % pid,
                'label': row['Target'],
                'boxes': []}

        # --- Add box if opacity is present
        if parsed[pid]['label'] == 1:
            parsed[pid]['boxes'].append(extract_box(row))

    return parsed

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR,'stage_2_train_labels.csv'))
df.head()
parsed = parse_data(df)

In [ ]:
img_list=list(parsed.keys())
print(len(img_list))

In [ ]:
class pneumoniaConfig(Config):
    """Configuration for training on the nucleus segmentation dataset."""
    # Give the configuration a recognizable name
    NAME = "pneumonia"

    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 8
    GPU_COUNT = 1
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + nucleus

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = 200

    DETECTION_MIN_CONFIDENCE = 0.78

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    BACKBONE = "resnet50"


    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)


    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    TRAIN_ROIS_PER_IMAGE = 50

    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 4

    # Max number of final detections per image
    DETECTION_MAX_INSTANCES = 3
    DETECTION_NMS_THRESHOLD = 0.01
class InferenceConfig(pneumoniaConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
config=pneumoniaConfig()
config.display()


In [ ]:
class pneumoniaDataset(utils.Dataset):
    """Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, image_list, parsed, h,w):
        super().__init__(self)
        
        # Add classes
        self.add_class('pneumonia', 1, 'Lung Opacity')
        
        # add images 
        for pid in image_list:
            boxes = parsed[pid]['boxes']
            self.add_image('pneumonia', image_id=pid, path=parsed[pid]['dicom'],boxes=boxes, 
                            h=h, w=w)
            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        d = pydicom.read_file(fp)
        im = d.pixel_array

    # --- Convert from single-channel grayscale to 3-channel RGB
        im = np.stack([im] * 3, axis=2)
        return im

    def load_mask(self, image_id):
#          Generate instance masks for an image.
#        Returns:
#         masks: A bool array of shape [height, width, instance count] with
#             one mask per instance.
#         class_ids: a 1D array of class IDs of the instance masks.
#         
        info = self.image_info[image_id]
        boxes = info['boxes']
        count = len(boxes)
        if count == 0:
            mask = np.zeros((info['h'], info['w'], 1), dtype=np.bool)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['h'], info['w'], count), dtype=np.bool)
            class_ids = np.ones((count,), dtype=np.int32)
            for i, box in enumerate(boxes):
                box = [int(b) for b in box]
                y1, x1, height, width = box
                y2 = y1 + height
                x2 = x1 + width
                mask[y1:y2, x1:x2 ,i]=1
        return mask, class_ids

In [ ]:
random.seed(42)
random.shuffle(img_list)
val = 1500
im_val = img_list[:val]
im_train = img_list[val:]

In [ ]:
# Training dataset.
dataset_train = pneumoniaDataset(im_train, parsed, 1024,1024)
dataset_train.prepare()

# Validation dataset
dataset_val = pneumoniaDataset(im_val,parsed, 1024,1024)
dataset_val.prepare()

In [ ]:
# Image augmentation (light but constant)
augmentation = iaa.Sequential([
    iaa.OneOf([ ## geometric transform
        iaa.Affine(
            scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
            translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
            rotate=(-2, 2),
            shear=(-1, 1),
        ),
        iaa.PiecewiseAffine(scale=(0.001, 0.025)),
    ]),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen
        iaa.GaussianBlur(sigma=(0.0, 0.1)),
        iaa.Sharpen(alpha=(0.0, 0.1)),
    ]),
])

# test on the same image as above
# imggrid = augmentation.draw_grid(image[:, :, 0], cols=5, rows=2)
# plt.figure(figsize=(30, 12))
# _ = plt.imshow(imggrid[:, :, 0], cmap='gray')

In [ ]:
# model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

# # Exclude the last layers because they require a matching
# # number of classes
# # model.load_weights("/kaggle/input/weights/mask_rcnn_pneumonia_0013.h5", by_name=True, exclude=[
# #     "mrcnn_class_logits", "mrcnn_bbox_fc",
# #     "mrcnn_bbox", "mrcnn_mask"])
# model.load_weights("/kaggle/input/latest/latest.h5", by_name=True)

In [ ]:
LEARNING_RATE = 0.006

# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# model.train(dataset_train, dataset_val,
#             learning_rate=LEARNING_RATE/5,
#             epochs=3,
#             layers='all',
#             augmentation=None)

# history = model.keras_model.history.history

In [ ]:
# model.train(dataset_train, dataset_val,
#             learning_rate=LEARNING_RATE/5,
#             epochs=6,
#             layers='all',
#             augmentation=None)
# history = model.keras_model.history.history
# # new_history = model.keras_model.history.history
# # for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
# model.keras_model.save_weights(ROOT_DIR+"/11.h5")

In [ ]:
# model.train(dataset_train, dataset_val,
#             learning_rate=LEARNING_RATE/5,
#             epochs=11,
#             layers='all',
#             augmentation=augmentation)
# # history = model.keras_model.history.history
# new_history = model.keras_model.history.history
# for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
# model.keras_model.save_weights(ROOT_DIR+"/21.h5")

In [ ]:
# model.train(dataset_train, dataset_val,
#             learning_rate=LEARNING_RATE/5,
#             epochs=16,
#             layers='all',
#             augmentation=augmentation)
# # history = model.keras_model.history.history
# new_history = model.keras_model.history.history
# for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
# model.keras_model.save_weights(ROOT_DIR+"/31.h5")

In [ ]:
# print(history)

In [ ]:
# model_path="/kaggle/input/weights/mask_rcnn_pneumonia_0013.h5"

In [ ]:
# epochs = range(1,len(next(iter(history.values())))+1)
# best_epoch = np.argmin(history["val_loss"])
# print("Best Epoch:", best_epoch + 1, history["val_loss"][best_epoch])

In [ ]:
# dir_names = next(os.walk(model.model_dir))[1]
# key = config.NAME.lower()
# dir_names = filter(lambda f: f.startswith(key), dir_names)
# dir_names = sorted(dir_names)

# if not dir_names:
#     import errno
#     raise FileNotFoundError(
#         errno.ENOENT,
#         "Could not find model directory under {}".format(self.model_dir))
    
# fps = []
# # Pick last directory
# for d in dir_names: 
#     dir_name = os.path.join(model.model_dir, d)
#     # Find the last checkpoint
#     checkpoints = next(os.walk(dir_name))[2]
#     checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
#     checkpoints = sorted(checkpoints)
#     if not checkpoints:
#         print('No weight files in {}'.format(dir_name))
#     else:
#         checkpoint = os.path.join(dir_name, checkpoints[best_epoch])
#         fps.append(checkpoint)

# model_path = sorted(fps)[-1]
# print('Found model {}'.format(model_path))

In [ ]:
# model_path=ROOT_DIR+"/3.h5"

In [ ]:
model_path="/kaggle/input/latest2/mask_rcnn_pneumonia_0011.h5"

In [ ]:
inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=ROOT_DIR)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
def predict(image_fps, filepath='submission.csv', min_conf=0.96):
    # assume square image
    resize_factor = 1024 / config.IMAGE_SHAPE[0]
    #resize_factor = ORIG_SIZE
    with open(filepath, 'w') as file:
        file.write("patientId,PredictionString\n")

        for image_id in tqdm(image_fps):
            ds = pydicom.read_file(image_id)
            image = ds.pixel_array
            # If grayscale. Convert to RGB for consistency.
            if len(image.shape) != 3 or image.shape[2] != 3:
                image = np.stack((image,) * 3, -1)
            image, window, scale, padding, crop = utils.resize_image(
                image,
                min_dim=config.IMAGE_MIN_DIM,
                min_scale=config.IMAGE_MIN_SCALE,
                max_dim=config.IMAGE_MAX_DIM,
                mode=config.IMAGE_RESIZE_MODE)

            patient_id = os.path.splitext(os.path.basename(image_id))[0]

            results = model.detect([image])
            r = results[0]

            out_str = ""
            out_str += patient_id
            out_str += ","
            assert( len(r['rois']) == len(r['class_ids']) == len(r['scores']) )
            if len(r['rois']) == 0:
                pass
            else:
                num_instances = len(r['rois'])

                for i in range(num_instances):
                    if r['scores'][i] > min_conf:
                        out_str += ' '
                        out_str += str(round(r['scores'][i], 2))
                        out_str += ' '

                        # x1, y1, width, height
                        x1 = r['rois'][i][1]
                        y1 = r['rois'][i][0]
                        width = r['rois'][i][3] - x1
                        height = r['rois'][i][2] - y1
                        bboxes_str = "{} {} {} {}".format(x1*resize_factor, y1*resize_factor, \
                                                           width*resize_factor, height*resize_factor)
                        out_str += bboxes_str

            file.write(out_str+"\n")
#             print(out_str)

In [ ]:
test_lis = glob.glob(test_dir+'/'+'*.dcm')
test_im=list(set(test_lis))
print(len(test_im))
# print(test_im)

In [ ]:
submission_path = os.path.join(ROOT_DIR, 'submission_final.csv')
predict(test_im, filepath=submission_path)
# print(submission_fp)

In [ ]:
submission_path2 = os.path.join(ROOT_DIR, 'submission_final2.csv')
predict(test_im, submission_path2,0.95)

In [ ]:
# from IPython.display import FileLink
# FileLink(model_path)

In [ ]:
# torch.save(model.state_dict(), ROOT_DIR+"/checkpoint.pth")

In [ ]:
# import keras
# import json

# def save_model(trained_model, out_fname="model.json"):
#     jsonObj = trained_model.keras_model.to_json()
#     with open(out_fname, "w") as fh:
#         fh.write(jsonObj)
#     fh.close()

# save_model(model, ROOT_DIR+"/mymodel.json")

In [ ]:
# model.keras_model.save_weights(ROOT_DIR+"/last_7.h5")

In [ ]:
# model.keras_model.save_weights(ROOT_DIR+"/last_7.h5")

In [ ]:
# json_file = open(ROOT_DIR+"/mymodel.json", 'r')

In [ ]:
# from keras.models import model_from_json
# loaded_model_json = json_file.read()
# json_file.close()

In [ ]:
# loaded_model = model_from_json(loaded_model_json)
# loaded_model.load_weights(ROOT_DIR+"/last_7.h5")